In [1]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

rides_previous_672_hour  rides_previous_671_hour  \
0                           1.0                      1.0   
1                           1.0                      0.0   
2                           0.0                      1.0   
3                           4.0                      2.0   
4                           0.0                      0.0   
...                         ...                      ...   
111211                      0.0                      0.0   
111212                      0.0                      0.0   
111213                      2.0                      0.0   
111214                      0.0                      0.0   
111215                      0.0                      0.0   

        rides_previous_670_hour  rides_previous_669_hour  \
0                           1.0                      0.0   
1                           1.0                      0.0   
2                           2.0                      0.0   
3                           2.0                      1.0   
4                           1.0                      0.0   
...                         ...                      ...   
111211                      1.0                      0.0   
111212                      0.0                      0.0   
111213                      0.0                      0.0   
111214                      1.0                      0.0   
111215                      0.0                      0.0   

        rides_previous_668_hour  rides_previous_667_hour  \
0                           1.0                      0.0   
1                           0.0                      0.0   
2                           0.0                      2.0   
3                           0.0                      0.0   
4                           0.0                      0.0   
...                         ...                      ...   
111211                      0.0                      0.0   
111212                      0.0                      0.0   
111213                      0.0                      0.0   
111214                      0.0                      2.0   
111215                      0.0                      2.0   

        rides_previous_666_hour  rides_previous_665_hour  \
0                           1.0                      0.0   
1                           0.0                      0.0   
2                           1.0                      1.0   
3                           0.0                      2.0   
4                           3.0                      1.0   
...                         ...                      ...   
111211                      0.0                      0.0   
111212                      0.0                      0.0   
111213                      0.0                      1.0   
111214                      0.0                      1.0   
111215                      0.0                      0.0   

        rides_previous_664_hour  rides_previous_663_hour  ...  \
0                           0.0                      0.0  ...   
1                           0.0                      1.0  ...   
2                           1.0                      2.0  ...   
3                           1.0                      4.0  ...   
4                           3.0                      2.0  ...   
...                         ...                      ...  ...   
111211                      1.0                      1.0  ...   
111212                      0.0                      0.0  ...   
111213                      1.0                      1.0  ...   
111214                      0.0                      0.0  ...   
111215                      1.0                      0.0  ...   

        rides_next_27_hour  rides_next_28_hour  rides_next_29_hour  \
0                      1.0                 0.0                 0.0   
1                      4.0                 0.0                 0.0   
2                      0.0                 0.0                 0.0   
3                      2.0                 1.0                 

In [2]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    targets_columns_names=[c for c in df.columns if c.startswith('rides_next_')]
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(40713, 674)
y_train.shape=(40713, 36)
X_test.shape=(70503, 674)
y_test.shape=(70503, 36)


In [3]:
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

In [4]:
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [6]:
model = MultiOutputRegressor(lgb.LGBMRegressor(force_col_wise=True))
model.fit(X_train_only_numeric, y_train)

[LightGBM] [Info] Total Bins 11332
[LightGBM] [Info] Number of data points in the train set: 40713, number of used features: 672
[LightGBM] [Info] Start training from score 0.416796
[LightGBM] [Info] Total Bins 11332
[LightGBM] [Info] Number of data points in the train set: 40713, number of used features: 672
[LightGBM] [Info] Start training from score 0.287181
[LightGBM] [Info] Total Bins 11332
[LightGBM] [Info] Number of data points in the train set: 40713, number of used features: 672
[LightGBM] [Info] Start training from score 0.155110
[LightGBM] [Info] Total Bins 11332
[LightGBM] [Info] Number of data points in the train set: 40713, number of used features: 672
[LightGBM] [Info] Start training from score 0.090856
[LightGBM] [Info] Total Bins 11332
[LightGBM] [Info] Number of data points in the train set: 40713, number of used features: 672
[LightGBM] [Info] Start training from score 0.081473
[LightGBM] [Info] Total Bins 11332
[LightGBM] [Info] Number of data points in the train se

MultiOutputRegressor(estimator=LGBMRegressor(force_col_wise=True))

In [7]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)
predictions

array([[1.96512204, 1.34368518, 0.4580491 , ..., 3.52564071, 3.02050863,
        2.60334855],
       [3.09683866, 1.10973857, 0.74087229, ..., 2.94756746, 3.19742519,
        2.56195655],
       [2.36815115, 0.71684488, 0.50819091, ..., 0.58837475, 1.17678882,
        1.08878355],
       ...,
       [0.38675463, 0.19354976, 0.11914225, ..., 0.48228109, 0.69480814,
        0.84868286],
       [0.62098773, 0.2599966 , 0.23951473, ..., 0.70585363, 0.61782189,
        0.34212027],
       [0.46078771, 0.28052014, 0.19415724, ..., 0.23471873, 0.31841618,
        0.18071773]])

In [8]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=}')

test_mae=0.6789886231199219


rides_next_1_hour  rides_next_2_hour  rides_next_3_hour  \
0                    1.0                0.0                2.0   
1                    2.0                0.0                1.0   
2                    2.0                0.0                0.0   
3                    3.0                0.0                1.0   
4                    1.0                2.0                0.0   
...                  ...                ...                ...   
70498                0.0                0.0                0.0   
70499                1.0                0.0                2.0   
70500                1.0                0.0                1.0   
70501                1.0                0.0                0.0   
70502                1.0                0.0                1.0   

       rides_next_4_hour  rides_next_5_hour  rides_next_6_hour  \
0                    3.0                0.0                0.0   
1                    0.0                0.0                0.0   
2                    0.0                0.0                0.0   
3                    0.0                0.0                0.0   
4                    0.0                0.0                0.0   
...                  ...                ...                ...   
70498                0.0                0.0                1.0   
70499                0.0                0.0                0.0   
70500                0.0                0.0                0.0   
70501                0.0                0.0                1.0   
70502                0.0                0.0                1.0   

       rides_next_7_hour  rides_next_8_hour  rides_next_9_hour  \
0                    0.0                5.0                3.0   
1                    1.0                1.0                4.0   
2                    2.0                1.0                2.0   
3                    0.0                0.0                0.0   
4                    0.0                0.0                0.0   
...                  ...                ...                ...   
70498                0.0                0.0                1.0   
70499                0.0                0.0                1.0   
70500                0.0                2.0                3.0   
70501                0.0                0.0                2.0   
70502                0.0                0.0                1.0   

       rides_next_10_hour  ...  rides_next_27_hour  rides_next_28_hour  \
0                     3.0  ...                 1.0                 0.0   
1                     2.0  ...                 0.0                 0.0   
2                     5.0  ...                 1.0                 0.0   
3                     0.0  ...                 0.0                 0.0   
4                     2.0  ...                 1.0                 0.0   
...                   ...  ...                 ...                 ...   
70498                 0.0  ...                 2.0                 0.0   
70499                 0.0  ...                 1.0                 0.0   
70500                 0.0  ...                 0.0                 0.0   
70501                 0.0  ...                 1.0                 0.0   
70502                 2.0  ...                 0.0                 0.0   

       rides_next_29_hour  rides_next_30_hour  rides_next_31_hour  \
0                     0.0                 0.0                 1.0   
1                     0.0                 0.0                 2.0   
2                     0.0                 0.0                 0.0   
3                     0.0                 0.0                 0.0   
4                     0.0                 0.0                 1.0   
...                   ...                 ...                 ...   
70498                 0.0                 0.0                 0.0   
70499                 0.0                 0.0                 0.0   
70500                 0.0                 1.0                 0.0   
70501                 0.0                 1.0    